In [ ]:
#@title Install and import necessary libraries
%pip install -qU tool-parse[langchain] langchain-ollama langgraph duckduckgo-search

from tool_parse.integrations.langchain import ExtendedStructuredTool, patch_chat_model

import typing as t
from duckduckgo_search import DDGS
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

#### Define tools

In [ ]:
async def search_text(
    text: str,
    *,
    safe_search: bool = True,
    backend: t.Literal["api", "html", "lite"] = "api",
    max_results: int = 1,
):
    """
    Search for text in the web.
    :param text: Text to search for.
    :param safe_search: If True, enable safe search.
    :param backend: Backend to use for retrieving results.
    :param max_results: Max results to return.
    """
    return DDGS().text(
        keywords=text,
        safesearch="on" if safe_search else "off",
        backend=backend,
        max_results=max_results,
    )

class ProductInfo(t.NamedTuple):  # Can be t.TypedDict or pydantic.BaseModel
    """
    Information about the product.
    :param name: Name of the product.
    :param price: Price of the product.
    :param in_stock: If the product is in stock.
    """

    name: str
    price: float
    in_stock: bool = False

tools = [
    ExtendedStructuredTool(func=search_text),
    ExtendedStructuredTool(func=ProductInfo, name="product_info"),
]
# OR # tools = ExtendedStructuredTool.from_objects(search_text, ProductInfo)

#### Patch the langchain chat model

In [ ]:
model = patch_chat_model(ChatOllama(model="llama3-groq-tool-use")) # Patch the instance
# OR
model = patch_chat_model(ChatOllama)(model="llama3-groq-tool-use") # Patch the class and then instantiate it

In [ ]:
model = patch_chat_model(ChatOllama)(model="llama3-groq-tool-use")
# OR # model = patch_chat_model(ChatOllama)(model="llama3-groq-tool-use")

agent_executor = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "abc123"}}

def query_agent(__query: str):
    for chunk in agent_executor.stream({"messages": [HumanMessage(content=__query)]}, config):
        message = list(chunk.values())[0]["messages"][0]
        if isinstance(message, AIMessage):
            print("[AGENT]")
            if message.tool_calls:
                print("Calling tool:")
                metadata = message.tool_calls[0]
                print(f"name={metadata['name']!r}")
                print(f"arguments={metadata['args']}")
            else:
                print(message.content)
        elif isinstance(message, ToolMessage):
            print(f"[{message.name!r} TOOL OUTPUT]")
            print(message.content)

        print("---" * 20)

query_agent("Search 5 sources for langgraph docs using lite backend")
query_agent("Parse: Product RTX 4900, priced at $3.5k, is in stock.")
